In [124]:
using BenchmarkTools

In [125]:
import Collective
import PuzzleTools

In [198]:
words = lowercase.(strip.(split(readstring("data/113809of.fic"))));

In [237]:
reload("Collective")

In [238]:
corpus = Collective.Corpus(words[1:Int(round(length(words) / 10000)):end]);

In [241]:
Collective.contains_repeated_consonant("carousels")

true

In [248]:
for i in 1:length(corpus.frequencies)
    if corpus.features.descriptions[i] == "contains 'l' at index 1"
        @show i corpus.frequencies[i]
        break
    end
end

i = 339
corpus.frequencies[i] = 0.03266647337392481


In [235]:
corpus.features.evaluators[1461].(["levitate", "inanimate", "sizes", "lightly", "crocodile", "legislator", "carousels"])

7-element Array{Bool,1}:
 true
 true
 true
 true
 true
 true
 true

In [244]:
sort(Collective.analyze(corpus, ["levitate", "inanimate", "sizes", "lightly", "crocodile", "legislator", "carousels"]))[1:20]

20-element Array{Collective.FeatureResult,1}:
 Collective.FeatureResult("contains 'l' at index 1",Collective.#26,Bool[true,false,false,true,false,true,false],0.00110508)           
 Collective.FeatureResult("has scrabble score 11",Collective.#24,Bool[true,true,false,false,false,true,true],0.00313566)              
 Collective.FeatureResult("contains 'c' at index 9 from end",Collective.#27,Bool[false,false,false,false,true,false,true],0.0079345)  
 Collective.FeatureResult("contains a repeated consonant",Collective.#35,Bool[true,true,true,true,true,true,true],0.00939951)         
 Collective.FeatureResult("contains 'l' at index 2 from end",Collective.#27,Bool[false,false,false,true,true,false,true],0.0095657)   
 Collective.FeatureResult("contains 'l' at index 8",Collective.#26,Bool[false,false,false,false,true,false,true],0.00987603)          
 Collective.FeatureResult("has 4 reverse alphabetical bigrams",Collective.#44,Bool[true,true,false,false,true,true,true],0.011867)    
 Collecti

In [121]:
sort(Collective.analyze(corpus, ["grimaced", "formally", "questionable", "discouraged", "communicated", "chrysalis", "saccharin"]))[1:5]

5-element Array{Collective.FeatureResult,1}:
 Collective.FeatureResult("contains a at index 4 from end",Bool[true,true,true,true,true,true,true],2.13491e-7)      
 Collective.FeatureResult("contains m at index 4",Bool[true,true,false,false,true,false,false],0.00185169)           
 Collective.FeatureResult("contains s at index 9 from end",Bool[false,false,true,true,false,false,true],0.00309857)  
 Collective.FeatureResult("5 unique vowels",Bool[false,false,true,true,true,false,false],0.00720193)                 
 Collective.FeatureResult("contains q at index 12 from end",Bool[false,false,true,false,false,false,false],0.0100677)

In [122]:
sort(Collective.analyze(corpus, ["thumbtacks", "monologue", "testimony", "camel", "meteorology", "trampoline", "achievement"]))[1:5]

5-element Array{Collective.FeatureResult,1}:
 Collective.FeatureResult("contains m",Bool[true,true,true,true,true,true,true],0.000187263)                        
 Collective.FeatureResult("contains t at index 1",Bool[true,false,true,false,false,true,false],0.00981769)          
 Collective.FeatureResult("contains h at index 9 from end",Bool[true,false,false,false,false,false,true],0.0184008) 
 Collective.FeatureResult("contains n at index 2 from end",Bool[false,false,true,false,false,true,true],0.0252534)  
 Collective.FeatureResult("contains t at index 10 from end",Bool[true,false,false,false,false,true,false],0.0285695)

In [123]:
word = "hello"
@benchmark $fset.evaluate($word)

BenchmarkTools.Trial: 
  memory estimate:  3.30 kb
  allocs estimate:  44
  --------------
  minimum time:     15.567 μs (0.00% GC)
  median time:      16.500 μs (0.00% GC)
  mean time:        19.378 μs (2.30% GC)
  maximum time:     4.539 ms (98.04% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%